In [ ]:
execution_time_full_features = []
execution_time_selected_features = []

In [1]:
import glob
import cv2
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler

import pickle
import time


# def import_images(folder_src):
#     path_images = glob.glob(folder_src+"*.jpg") #input path image source
#     total_faces = int(len(path_images) / 6)
#     for i in range(1, total_faces+1):
#         images = []
#         print(images)
#         path_image = glob.glob("Images/Negatif/*"+str(i)+".jpg")
#         for image in path_image:
#             filename = os.path.basename(image[:-4])
#             image = cv2.imread(image)
#             gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#             images.append(gray)
#             print(filename)
        


path_image = glob.glob("Images/Negatif/*1.jpg")

def import_images(path_image):
  images = []
  for image in path_image:
      filename = os.path.basename(image[:-4])
      image = cv2.imread(image)
      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      images.append(gray)
      print(filename)

  return images
# folder_src = "Images/Negatif/"
images = import_images(path_image)


Bridge Nose1
Forehead1
Left Canthus1
Left Crowsfeet1
Right Canthus1
Right Crowsfeet1


In [2]:
import numpy as np

# First Order Statistic
def FirstOrderStatistic(f, mask):
    '''
    Parameters
    ----------
    f : numpy ndarray
        Image of dimensions N1 x N2.
    mask : numpy ndarray
        Mask image N1 x N2 with 1 if pixels belongs to ROI, 0 else. Give None
        if you want to consider ROI the whole image.

    Returns
    -------
    features : numpy ndarray
        1)Mean, 2)Variance, 3)Median (50-Percentile), 4)Mode, 
        5)Skewness, 6)Kurtosis, 7)Energy, 8)Entropy, 
        9)Minimal Gray Level, 10)Maximal Gray Level, 
        11)Coefficient of Variation, 12,13,14,15)10,25,75,90-
        Percentile, 16)Histogram width
    labels : list
        Labels of features.
    '''
    if mask is None:
        mask = np.ones(f.shape)
    
    # 1) Labels
    labels = ["FOS_Mean","FOS_Mode","FOS_Median","FOS_Variance","FOS_StandardDeviation","FOS_Skewness",
              "FOS_Kurtosis","FOS_Energy","FOS_Entropy","FOS_MinimalGrayLevel",
              "FOS_MaximalGrayLevel","FOS_CoefficientOfVariation",
              "FOS_10Percentile","FOS_25Percentile","FOS_75Percentile",
              "FOS_90Percentile","FOS_HistogramWidth","FOS_Smoothness"]
    
    # 2) Parameters
    f  = f.astype(np.uint8)
    mask = mask.astype(np.uint8)
    level_min = 0
    level_max = 255
    Ng = (level_max - level_min) + 1
    bins = Ng
    
    # 3) Calculate Histogram H inside ROI
    f_ravel = f.ravel() 
    mask_ravel = mask.ravel() 
    roi = f_ravel[mask_ravel.astype(bool)] 
    H = np.histogram(roi, bins=bins, range=[level_min, level_max], density=True)[0]
    
    # 4) Calculate Features
    features = np.zeros(18,np.double)  
    i = np.arange(0,bins)
    features[0] = np.dot(i,H) #mean
    features[1] = np.argmax(H) #mode
    features[2] = np.percentile(roi,50)  #median
    features[3] = sum(np.multiply(((i-features[0])**2),H)) #variance
    features[4] = np.sqrt(features[1]) #std dev
    features[5] = sum(np.multiply(((i-features[0])**3),H))/(np.sqrt(features[1])**3) #skewness
    features[6] = sum(np.multiply(((i-features[0])**4),H))/(np.sqrt(features[1])**4) #kurtosis
    features[7] = sum(np.multiply(H,H)) #energy
    features[8] = -sum(np.multiply(H,np.log(H+1e-16))) # entropy
    features[9] = min(roi) # min
    features[10] = max(roi) # max
    features[11] = np.sqrt(features[2]) / features[0] # coef var
    features[12] = np.percentile(roi,10) 
    features[13] = np.percentile(roi,25)  
    features[14] = np.percentile(roi,75) 
    features[15] = np.percentile(roi,90) 
    features[16] = features[14] - features[11] # hist width
    features[17] = 1-(1/(1+features[2])) # smoothness
    
    return features, labels

In [3]:
from pyfeats import *
from _mahotas.features import texture

# Mask is None
mask = None

# Array to contain the result
# 1st order statistics
a_features_fos=[]
# glcm
a_features_mean_glcm =[]
a_features_range_glcm = []
# glds
a_features_glds = [] 
# glrlm
a_features_glrlm = []

# texture features extraction
def feature_extraction(images, mask):
    for f in images:
        #extract fos feature
        features_fos, labels_fos = FirstOrderStatistic(f, mask) 
        a_features_fos.append(features_fos)
        
        #ekstrak fitur tekstur glcm
        features_mean_glcm, features_range_glcm, labels_mean_glcm, labels_range_glcm = glcm_features(f, ignore_zeros=True)
        a_features_mean_glcm.append(features_mean_glcm)
        a_features_range_glcm.append(features_range_glcm)

        #ekstrak fitur tekstur glds
        features_glds, labels_glds = glds_features(f, mask, Dx=[0,1,1,1], Dy=[1,1,0,-1]) 
        a_features_glds.append(features_glds) 

        # #ekstrak fitur tekstur glrlm
        features_glrlm, labels_glrlm = glrlm_features(f, mask, Ng=256)
        a_features_glrlm.append(features_glrlm)
    return labels_fos,labels_mean_glcm, labels_range_glcm, labels_glds, labels_glrlm 

In [4]:
# GLCM Library
import mahotas

def extract_haralick(images):
    distance = 5 # default value is 1
    features = []
    for image in images:
        haralick = mahotas.features.haralick(image, distance = distance, ignore_zeros=True, compute_14th_feature=True,)
        haralick1 = np.hstack(np.hstack(haralick))
        features.append(haralick1)
    return features

In [5]:
labels_fos, labels_mean_glcm, labels_range_glcm, labels_glds, labels_glrlm = feature_extraction(images, mask)
features = extract_haralick(images)

# Naming Header Column of GLCMC four dir
angles = ['0', '45', '90','135']
haralick = ["ASM", "contrast", "correlation", "ssvar", "IDM", "SAV",
            "SEntro","Svar","entropy","Dvar", "DEntro","IMCorr1","IMCorr2", "MCC"]
columns = []
for n in haralick:
    for a in angles:
        columns.append(n + "_" + a)

# Fos
df_fos = pd.DataFrame(a_features_fos, columns=labels_fos)
# GLCM
df_glcm = pd.DataFrame(a_features_mean_glcm, columns=labels_mean_glcm)
df_GLCM_four_dir = pd.DataFrame(features, columns=columns)

# GLDS
df_glds = pd.DataFrame(a_features_glds, columns=labels_glds)
# GLRLM
df_glrlm = pd.DataFrame(a_features_glrlm, columns=labels_glrlm)

df_result = pd.concat([df_fos, df_glcm, df_GLCM_four_dir, df_glds, df_glrlm], axis=1)
df_result

,FOS_Mean,FOS_Mode,FOS_Median,FOS_Variance,FOS_StandardDeviation,FOS_Skewness,FOS_Kurtosis,FOS_Energy,FOS_Entropy,FOS_MinimalGrayLevel,...,GLRLM_LongRunEmphasis,GLRLM_GrayLevelNo-Uniformity,GLRLM_RunLengthNonUniformity,GLRLM_RunPercentage,GLRLM_LowGrayLevelRunEmphasis,GLRLM_HighGrayLevelRunEmphasis,GLRLM_Short owGrayLevelEmphasis,GLRLM_ShortRunHighGrayLevelEmphasis,GLRLM_LongRunLowGrayLevelEmphasis,GLRLM_LongRunHighGrayLevelEmphasis
0,106.040703,121.0,105.0,736.444466,11.000000,-0.761302,92.788978,0.010445,4.709543,16.0,...,1.557470,1998.264297,150721.419692,1.603554,1.143658e-04,12010.797111,1.010767e-04,11069.152022,1.0,17171.180018
1,104.659923,102.0,103.0,62.209089,10.099505,0.236865,1.492932,0.039543,3.452741,60.0,...,1.455425,8837.489018,183144.244008,1.764316,9.506549e-05,10765.259194,8.654628e-05,9853.077605,1.0,15489.535742
2,57.048531,61.0,58.0,427.006885,7.810250,-6.108032,167.449184,0.014928,4.399717,1.0,...,1.882273,866.068900,42261.992290,0.858992,5.509891e+11,3644.180926,5.509891e+11,3237.303781,1.0,6262.934600
3,81.252887,94.0,86.0,584.531457,9.695360,-17.039605,152.450201,0.015042,4.446750,5.0,...,2.127802,712.774059,32183.511321,0.764913,4.763633e-04,6935.522184,3.733662e-04,5772.050623,1.0,14870.170317
4,68.278004,56.0,67.0,744.510114,7.483315,4.407219,514.827981,0.010699,4.692788,5.0,...,1.524945,512.898678,38534.660844,0.808055,5.442472e-04,5398.990599,4.610304e-04,5015.421903,1.0,7453.995665
5,128.680157,149.0,141.0,1009.741487,12.206556,-28.779907,221.356651,0.019042,4.415372,15.0,...,1.570343,969.317274,42413.372682,0.852782,1.150459e-04,16986.599762,1.040172e-04,15186.365979,1.0,27581.265891


In [10]:
X = df_result

sc = StandardScaler()
scaler = sc.fit(X)
X_scaled = scaler.transform(X)


array([[ 6.14594510e-01,  7.36048315e-01,  4.26132714e-01,
         4.80585633e-01,  7.71680179e-01,  6.30983188e-01,
        -6.18806065e-01, -7.88267230e-01,  8.45555372e-01,
        -5.00208464e-02,  2.33989314e-01, -8.26349683e-01,
         4.78932319e-01,  3.12457048e-01,  7.35252643e-01,
         1.03966227e+00,  7.35337531e-01,  6.27232358e-01,
        -1.13124698e+00,  8.90865613e-01,  3.68262862e-01,
         4.77764320e-01, -5.80667777e-01,  6.16987101e-01,
         4.71200630e-01,  8.36144996e-01,  1.13024941e+00,
        -5.68235617e-01,  8.16471619e-01, -1.08073084e-01,
         4.47162430e-01,  1.02603770e+00, -1.04824677e+00,
         3.16626938e-01,  4.46352805e-01,  4.76747052e-01,
        -2.61743066e-01,  6.23683280e-01,  4.72174401e-01,
         8.01961585e-01,  9.78097156e-01, -4.34163982e-01,
         4.64100320e-01, -3.46700421e-01,  4.68029166e-01,
        -2.18675503e-02, -1.00317695e+00, -6.59712206e-02,
         4.89040641e-01,  4.60117474e-01, -1.61344734e-0

In [7]:
start_time1 = time.time()
# Full Features
clf_full = pickle.load(open(r'Backup\4. Hasil Pengujian 4 Agustus 2023\Save Model\Full Features.pkl', 'rb'))
predictions1 = clf_full.predict(X_scaled)
print(predictions1)

ans1 = sum(predictions1)

if ans1 < 3:
    print("Risiko Rendah PJK")
else:
    print("Risiko Tinggi PJK")
# End

end_time1 = time.time()
execution_time1 = end_time1 - start_time1
print("Execution time:",execution_time1)

[0 0 1 1 0 0]
Risiko Rendah PJK
Execution time: 0.4743626117706299


In [9]:
cols = [2,  7,  8,  9, 10, 11, 13, 15, 17, 20, 23, 25, 29, 31, 35, 41, 42,
       44, 45, 47, 49, 51, 54, 56, 64, 69, 71, 73, 77, 78, 82, 83, 88, 90,
       94, 95, 96, 98, 99]

X_scaled2 = X_scaled.iloc[:, cols]

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [8]:
start_time2 = time.time()

# Selected Features
clf_select = pickle.load(open(r'Backup\4. Hasil Pengujian 4 Agustus 2023\Save Model\Subset Features.pkl', 'rb'))
predictions2 = clf_full.predict(X_scaled)
print(predictions2)

ans2 = sum(predictions2)

if ans2 < 3:
  print("Negatif")
else:
  print("Positif")
# End

end_time2 = time.time()
execution_time2 = end_time2 - start_time2 
print("Execution time:",execution_time2)

[0 0 1 1 0 0]
Negatif
Execution time: 0.004998683929443359
